In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from sklearn import preprocessing
#from lib.utils import split_test_train

import requests as r


In [ ]:
df[df.index == datetime(day=28, month=5,year=2021)]

In [ ]:
trades = [{
    'no_shares': 168.0000,
    'buy_price': 29.63,
    'ticker': 'VUKE.L',
    'date': datetime(month=3, day=2, year=2021)
},
{
    'no_shares': 58,
    'buy_price': 34.01,
    'ticker': 'VMID.L',
    'date': datetime(month=3, day=2, year=2021)
}
,{
    'no_shares': 1,
    'buy_price': 57.03,
    'ticker': 'VUSA.L',
    'date':datetime(day=14, month=4, year=2021)
}
,{
    'no_shares': 9,
    'buy_price': 57.03,
    'ticker': 'VUSA.L',
    'date':datetime(day=15,month=4,year=2021)
}
,{
    'no_shares': 10,
    'buy_price': 57.49,
    'ticker': 'VUSA.L',
    'date':datetime(day=16,month=4,year=2021)
}
,{
    'no_shares': 14,
    'buy_price': 491.60/14.0,
    'ticker': 'VMID.L',
    'date': datetime(day=20, month=5, year=2021)
}
,{
    'no_shares': 16,
    'buy_price': 495.43/16.0,
    'ticker': 'VUKE.L',
    'date': datetime(day=20, month=5, year=2021)
},
{
    'no_shares': 39,
    'buy_price': 995.90/39,
    'ticker': 'VJPN.L',
    'date':datetime(day=25, month=5,year=2021)
},
{
    'no_shares': 1,
    'buy_price': 25.495/1,
    'ticker': 'VJPN.L',
    'date':datetime(day=25, month=5,year=2021)
}
]


def print_trade_returns(trade, priceType='regularMarketPrice'):
    stock = yf.Ticker(trade.get('ticker'))
    price = stock.get_info()
    price = float(price.get(priceType))
    value = price * trade.get('no_shares')
    returns = (price - trade.get('buy_price'))/price
    print(f'{trade.get("ticker")} value: {value}, return: {returns*100}% cost={trade["no_shares"]*trade["buy_price"]}')
    return value

pxType='previousClose'
pxType='regularMarketPrice'
nav = 8.57
for trade in trades:
    nav += print_trade_returns(trade, pxType)
print(nav)


In [ ]:
tickers = yf.Tickers(['VUKE.L'])

In [ ]:
stock_list = {
    'VUKE.L': 'FTSE 100',
    'VMID.L': 'FTSE 250',
    'VUSA.L': 'S&P 500',
    'VERX.L': 'DEVELOPED EUROPE EX UK',
    'VGER.L': 'GERMANY ALL CAP',
    'VWRL.L': 'FTSE ALL WORLD',
    'VHYL.L': 'WORLD HIGH DIVIDEND YIELD',
    'VEVE.L': 'DEVELOPED WORLD',
    'VJPN.L': 'FTSE JAPAN',
    'VAPX.L': 'ASIA EX JAPAN',
    'VNRT.L': 'FTSE NORTH AMERICA',
    'VFEM.L': 'EMERGING MARKETS',
    'VECP.L': 'fi',
    'VETY.L': 'fi',
    'VAGP.L': 'fi',
    'VGOV.L': 'fi',
    'VUCP.L': 'fi',
    'VUTY.L': 'fi',
    'VEMT.L': 'fi'
}
#stock_list = ['VUKE.L', 'VMID.L', 'VUSA.L', 'VERX.L', 'VEUR.L', 'VGER.L', 'V3AM.L', 'VWRL.L', 'VEVE.L', 'VJPN.L','VAPX.L', 'VNRT.L', 'VFEM.L']
fixed_income = []

In [ ]:
yf.Ticker('VUKE.L').history('5y')

In [ ]:
stocks = yf.Tickers(list(stock_list.keys()))
stock_prices = stocks.history('5y')['Close']
#vuke = vuke.history('5y')

In [ ]:
df = stock_prices[list(filter(lambda col: len(stock_prices[stock_prices[col].notna()]) > 1000, stock_prices.columns))].dropna()

In [ ]:
hist = stocks.tickers['VUKE.L'].history('5y')

In [ ]:
req = r.get('https://covid.ourworldindata.org/data/owid-covid-data.csv')
with open('./data/covid-vaccination-data.csv', 'w') as fil: 
    fil.write(req.text)
covid = pd.read_csv('./data/covid-vaccination-data.csv')

In [ ]:
covid.date = pd.to_datetime(covid.date)

covid_grouped = covid.groupby('iso_code')

In [ ]:
covid = covid.set_index(pd.MultiIndex.from_frame(covid[['iso_code','date']]))

In [ ]:
covid.columns

In [ ]:
covid.new_cases_smoothed['JPN'].plot(label='JPN', legend=True)
covid.new_cases_smoothed['GBR'].plot(label='GBR', legend=True)

In [ ]:
#covid.reproduction_rate['IND'].plot(label='IND', legend=True)
#covid.reproduction_rate['GBR'].plot(label='GBR', legend=True)
#covid.new_cases_per_million['GER'].plot(label='GER', legend=True)
#covid.new_cases_per_million['USA'].plot()
#covid.new_cases_per_million['JPN'].plot()
stock_prices#.join(other=covid.reproduction_rate['IND'])

In [ ]:
df = stock_prices[list(filter(lambda col: len(stock_prices[stock_prices[col].notna()]) > 1000, stock_prices.columns))].dropna()
# rebase everything to 100
#from lib.utils import cumulative_returns, daily_returns

df_subset = df.loc[df.index > datetime(year=2020, month=3, day=1)]

for col in df_subset.columns:
    ser = pd.Series(daily_returns(df_subset[col]).replace(np.inf, 0))
    df_subset.loc[:,col] = ser
    df_subset.loc[:,f'c_{col}'] = cumulative_returns(ser)

#df_subset = df_subset.join(other=covid.reproduction_rate['GBR'])

df_subset[list(filter(lambda col: 'c_' in col, df_subset.columns))].plot(figsize=(15,7))

In [ ]:
sns.jointplot(data=df_subset, x="VJPN.L", y="VEUR.L")

In [ ]:
import requests as r
req = r.get('https://www.ft.com/search?q=FTSE100')


In [ ]:
import lxml.etree as ET
url = 'http://www.ft.com/search?q=FTSE100'

ET.fromstring(req.text)

In [ ]:
import psycopg2 as psql
import pandas as pd
con = psql.connect(host="localhost", port='5432', user='feeds', password='feeds', database='feeds')

q="""
SELECT * /*distinct table_name, table_schema*/
  FROM information_schema.tables
  where table_name like "t_stg_ArticleSummary_results";
"""

q="""select * from t_stg_ArticleSummary_results;"""

pd.read_sql_query(q, con)

#con.cursor().execute(q)

#print(psql.connect.__doc__)

In [ ]:


from sklearn.linear_model import Lasso


split_test_train(y=stock_prices['VUKE.L'],
                 X=stock_prices[list(filter(lambda col: col != 'VUKE.L', stock_prices.columns))])


In [ ]:
df[['VAPX.L', 'VUKE.L']]

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression

tscv = TimeSeriesSplit(max_train_size=3*365, n_splits=2)

features = list(filter(lambda col: 'c_' not in col and 'VUKE.L' not in col, df.columns))


for train_index, test_index in tscv.split(stock_prices['VUKE.L']):
    model = LinearRegression()
    model = model.fit(y=df['VUKE.L'].values[train_index], X=df[features].values[train_index])
    print(model)
    print(model.score(y=df['VUKE.L'].values[test_index], X=df[features].values[test_index]))


In [ ]:
df[['VJPN.L', 'VUKE.L', 'VMID.L']].plot()

In [ ]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import pandas as pd


def optimal_portfolio(returns):
    n = len(returns)
    returns = np.asmatrix(returns)
    
    N = 100
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis=1))
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] 
                  for mu in mus]
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    # CALCULATE THE OPTIMAL PORTFOLIO
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    return np.asarray(wt), returns, risks

weights, returns, risks = optimal_portfolio(df)

fig = plt.figure()
plt.plot(stds, means, 'o')
plt.ylabel('mean')
plt.xlabel('std')
plt.plot(risks, returns, 'y-o')
py.iplot_mpl(fig, filename='efficient_frontier', strip_style=True)
